# CAA Baseline Steering Methods
Three variants of Contrastive Activation Addition for steering language models:
1. **CAAMeanDiff** — Mean difference of class-conditional activations
2. **CAAContrastive** — Mean difference from paired contrastive prompts  
3. **CAARepE** — Representation Engineering with persona prefixes

We run each method on GPT-2 Small → Gemma-2-2B, evaluating steering effectiveness via probe score shift, KL divergence, and downstream classification.

In [ ]:
# @title Setup
import subprocess, sys, os

# Install dependencies (uncomment on Colab)
# !pip install -q torch transformer-lens sae-lens cvxpy datasets scikit-learn matplotlib seaborn tqdm transformers

# Add project root to path
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    # Adjust this path to where you uploaded/mounted the project
    PROJECT_ROOT = '/content/optimal_sparse_steering'
else:
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

sys.path.insert(0, PROJECT_ROOT)

import logging
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from configs.base import ExperimentConfig, ModelConfig, DataConfig
from src.models.wrapper import ModelWrapper
from src.data.loader import load_dataset_splits
from src.data.preprocessing import extract_activations
from src.data.prompts import get_contrastive_pairs, get_persona_prefix, get_neutral_queries
from src.probes.linear_probe import LinearProbe
from src.steering.caa import CAAMeanDiff, CAAContrastive, CAARepE
from src.evaluation.metrics import compute_probe_score, compute_kl_divergence
from src.evaluation.generation import steered_generation

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(name)s: %(message)s')
sns.set_theme(style="whitegrid")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## Helper Functions

In [ ]:
def evaluate_steering(model_wrapper, probe, test_activations, test_labels,
                      method, layer, alpha_values, target_class=1,
                      test_texts=None, n_gen=5):
    """Evaluate a steering method across multiple alpha values."""
    results = []
    sv = method.steering_vector.numpy()
    
    for alpha in alpha_values:
        steered_acts = test_activations + alpha * sv[np.newaxis, :]
        base_score = compute_probe_score(probe, test_activations, target_class)
        steered_score = compute_probe_score(probe, steered_acts, target_class)
        
        result = {
            'alpha': alpha,
            'probe_score_base': base_score,
            'probe_score_steered': steered_score,
            'probe_score_delta': steered_score - base_score,
            'steering_norm': float(method.steering_vector.norm().item()),
        }
        
        # Generate a few samples at this alpha
        if test_texts is not None and alpha in [0.0, 3.0, 5.0, 10.0]:
            queries = get_neutral_queries("sst2")[:n_gen]
            try:
                generated = steered_generation(
                    model_wrapper, queries, method, layer, alpha=alpha,
                    max_new_tokens=50, temperature=0.7, top_p=0.9,
                )
                result['generations'] = generated
            except Exception as e:
                result['generations'] = [f"Error: {e}"]
        
        results.append(result)
    
    return results


def plot_alpha_sweep(results_dict, title="Steering Effectiveness vs Alpha"):
    """Plot probe score delta across alpha values for multiple methods."""
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    
    for method_name, results in results_dict.items():
        alphas = [r['alpha'] for r in results]
        deltas = [r['probe_score_delta'] for r in results]
        ax.plot(alphas, deltas, marker='o', label=method_name)
    
    ax.set_xlabel('Alpha (steering strength)')
    ax.set_ylabel('Probe Score Delta')
    ax.set_title(title)
    ax.legend()
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
    return fig

---
## GPT-2 Small + SST-2

In [ ]:
# @title Load GPT-2 Small
config = ExperimentConfig()  # defaults to GPT-2 Small + SST-2
config.model.device = device

model_wrapper = ModelWrapper(config.model)
print(f"Model: {config.model.name}, d_model: {config.model.d_model}, layers: {config.model.n_layers}")

In [ ]:
# @title Load Data + Train Probe
data = load_dataset_splits(config.data)
print(f"Train: {len(data['train_texts'])}, Test: {len(data['test_texts'])}")

layer = config.model.steering_layer
print(f"\nExtracting activations at layer {layer}...")

train_acts = extract_activations(data['train_texts'], model_wrapper, layer,
                                  batch_size=config.model.batch_size)
test_acts = extract_activations(data['test_texts'], model_wrapper, layer,
                                 batch_size=config.model.batch_size)

train_acts_np = train_acts.numpy()
test_acts_np = test_acts.numpy()
train_labels = np.array(data['train_labels'])
test_labels = np.array(data['test_labels'])

# Train linear probe
probe = LinearProbe(d_model=config.model.d_model)
probe.fit(train_acts_np, train_labels)
print(f"Probe accuracy \u2014 train: {probe.score(train_acts_np, train_labels):.4f}, "
      f"test: {probe.score(test_acts_np, test_labels):.4f}")

### Method 1: CAAMeanDiff
Simple mean difference of class-conditional activations. The steering vector is $\hat{v} = \frac{\mu_+ - \mu_-}{\|\mu_+ - \mu_-\|}$

In [ ]:
# @title CAAMeanDiff
target_class = 1
pos_mask = train_labels == target_class
neg_mask = ~pos_mask

caa_md = CAAMeanDiff()
caa_md.compute_steering_vector(
    activations_pos=torch.tensor(train_acts_np[pos_mask]),
    activations_neg=torch.tensor(train_acts_np[neg_mask]),
)

print(f"Steering vector norm: {caa_md.steering_vector.norm():.4f}")
print(f"Steering vector shape: {caa_md.steering_vector.shape}")

alpha_values = [0.0, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0, 15.0, 20.0]
results_md = evaluate_steering(model_wrapper, probe, test_acts_np, test_labels,
                                caa_md, layer, alpha_values, target_class,
                                test_texts=data['test_texts'])

for r in results_md:
    print(f"  alpha={r['alpha']:5.1f}  probe_delta={r['probe_score_delta']:+.4f}")

### Method 2: CAAContrastive
Runs paired contrastive prompts through the model and takes the mean difference of activations at the steering layer.

In [ ]:
# @title CAAContrastive
pairs = get_contrastive_pairs("sst2")
print(f"Using {len(pairs)} contrastive pairs")
print(f"Example: (+) {pairs[0][0][:60]}...")
print(f"         (-) {pairs[0][1][:60]}...")

caa_con = CAAContrastive()
caa_con.compute_steering_vector(
    model_wrapper=model_wrapper, layer=layer,
    contrastive_pairs=pairs, batch_size=config.model.batch_size,
)

print(f"\nSteering vector norm: {caa_con.steering_vector.norm():.4f}")

# Cosine similarity with MeanDiff vector
cos_sim = torch.nn.functional.cosine_similarity(
    caa_md.steering_vector.unsqueeze(0),
    caa_con.steering_vector.unsqueeze(0)
).item()
print(f"Cosine similarity with MeanDiff: {cos_sim:.4f}")

results_con = evaluate_steering(model_wrapper, probe, test_acts_np, test_labels,
                                 caa_con, layer, alpha_values, target_class,
                                 test_texts=data['test_texts'])

for r in results_con:
    print(f"  alpha={r['alpha']:5.1f}  probe_delta={r['probe_score_delta']:+.4f}")

### Method 3: CAARepE (Representation Engineering)
Prepends positive/negative persona prefixes to neutral queries and computes the mean activation difference.

In [ ]:
# @title CAARepE
queries = get_neutral_queries("sst2")[:20]
pos_prefix = get_persona_prefix("sst2", 1)
neg_prefix = get_persona_prefix("sst2", 0)
print(f"Positive prefix: {pos_prefix[:80]}...")
print(f"Negative prefix: {neg_prefix[:80]}...")
print(f"Using {len(queries)} neutral queries")

caa_repe = CAARepE()
caa_repe.compute_steering_vector(
    model_wrapper=model_wrapper, layer=layer,
    neutral_queries=queries, positive_prefix=pos_prefix,
    negative_prefix=neg_prefix, batch_size=config.model.batch_size,
)

print(f"\nSteering vector norm: {caa_repe.steering_vector.norm():.4f}")

# Cosine similarities
for name, other in [("MeanDiff", caa_md), ("Contrastive", caa_con)]:
    cos = torch.nn.functional.cosine_similarity(
        caa_repe.steering_vector.unsqueeze(0),
        other.steering_vector.unsqueeze(0)
    ).item()
    print(f"Cosine similarity with {name}: {cos:.4f}")

results_repe = evaluate_steering(model_wrapper, probe, test_acts_np, test_labels,
                                  caa_repe, layer, alpha_values, target_class,
                                  test_texts=data['test_texts'])

for r in results_repe:
    print(f"  alpha={r['alpha']:5.1f}  probe_delta={r['probe_score_delta']:+.4f}")

### GPT-2 Small: Comparison

In [ ]:
# @title GPT-2 Small — CAA Comparison
all_results_gpt2 = {
    'CAAMeanDiff': results_md,
    'CAAContrastive': results_con,
    'CAARepE': results_repe,
}

fig = plot_alpha_sweep(all_results_gpt2, title="GPT-2 Small: CAA Steering Effectiveness")

# Print generated samples at alpha=5.0
print("\n" + "="*80)
print("Generated Samples (alpha=5.0)")
print("="*80)
for method_name, results in all_results_gpt2.items():
    for r in results:
        if r['alpha'] == 5.0 and 'generations' in r:
            print(f"\n--- {method_name} ---")
            for i, gen in enumerate(r['generations'][:3]):
                print(f"  [{i+1}] {gen[:150]}...")

---
## Gemma-2-2B Pretrained + SST-2

In [ ]:
# @title Load Gemma-2-2B (pretrained)
# Free up GPT-2 memory
del model_wrapper
torch.cuda.empty_cache()

gemma_model_config = ModelConfig(
    name="gemma-2-2b-pt",
    tl_name="google/gemma-2-2b",
    sae_release="gemma-scope-2b-pt-res-canonical",
    sae_id_template="layer_{layer}/width_16k/canonical",
    hook_template="blocks.{layer}.hook_resid_post",
    d_model=2304,
    n_layers=26,
    steering_layer=15,
    dtype="float16",  # float16 to fit on T4
    device=device,
    batch_size=4,
)

gemma_config = ExperimentConfig(
    model=gemma_model_config,
    experiment_name="gemma-2-2b-pt_sst2",
)

model_wrapper_gemma = ModelWrapper(gemma_config.model)
print(f"Model: {gemma_config.model.name}, d_model: {gemma_config.model.d_model}")

In [ ]:
# @title Gemma: Data + Probe
data_gemma = load_dataset_splits(gemma_config.data)

layer_gemma = gemma_config.model.steering_layer
print(f"Extracting activations at layer {layer_gemma}...")

train_acts_g = extract_activations(data_gemma['train_texts'], model_wrapper_gemma,
                                    layer_gemma, batch_size=gemma_config.model.batch_size)
test_acts_g = extract_activations(data_gemma['test_texts'], model_wrapper_gemma,
                                   layer_gemma, batch_size=gemma_config.model.batch_size)

train_acts_g_np = train_acts_g.numpy()
test_acts_g_np = test_acts_g.numpy()
train_labels_g = np.array(data_gemma['train_labels'])
test_labels_g = np.array(data_gemma['test_labels'])

probe_gemma = LinearProbe(d_model=gemma_config.model.d_model)
probe_gemma.fit(train_acts_g_np, train_labels_g)
print(f"Probe accuracy \u2014 train: {probe_gemma.score(train_acts_g_np, train_labels_g):.4f}, "
      f"test: {probe_gemma.score(test_acts_g_np, test_labels_g):.4f}")

In [ ]:
# @title Gemma: Run CAA Methods
target_class = 1
pos_mask_g = train_labels_g == target_class
neg_mask_g = ~pos_mask_g

# MeanDiff
caa_md_g = CAAMeanDiff()
caa_md_g.compute_steering_vector(
    activations_pos=torch.tensor(train_acts_g_np[pos_mask_g]),
    activations_neg=torch.tensor(train_acts_g_np[neg_mask_g]),
)

# Contrastive
caa_con_g = CAAContrastive()
caa_con_g.compute_steering_vector(
    model_wrapper=model_wrapper_gemma, layer=layer_gemma,
    contrastive_pairs=get_contrastive_pairs("sst2"),
    batch_size=gemma_config.model.batch_size,
)

# RepE
caa_repe_g = CAARepE()
caa_repe_g.compute_steering_vector(
    model_wrapper=model_wrapper_gemma, layer=layer_gemma,
    neutral_queries=get_neutral_queries("sst2")[:20],
    positive_prefix=get_persona_prefix("sst2", 1),
    negative_prefix=get_persona_prefix("sst2", 0),
    batch_size=gemma_config.model.batch_size,
)

# Evaluate
alpha_values_g = [0.0, 1.0, 3.0, 5.0, 10.0, 15.0]
all_results_gemma = {}
for name, method in [('CAAMeanDiff', caa_md_g), ('CAAContrastive', caa_con_g), ('CAARepE', caa_repe_g)]:
    results = evaluate_steering(model_wrapper_gemma, probe_gemma, test_acts_g_np,
                                 test_labels_g, method, layer_gemma, alpha_values_g, target_class)
    all_results_gemma[name] = results
    print(f"\n{name}:")
    for r in results:
        print(f"  alpha={r['alpha']:5.1f}  probe_delta={r['probe_score_delta']:+.4f}")

In [ ]:
# @title Gemma: Comparison Plot
fig = plot_alpha_sweep(all_results_gemma, title="Gemma-2-2B: CAA Steering Effectiveness")

---
## Cross-Model Summary

In [ ]:
# @title Cross-Model Summary
import pandas as pd

rows = []
for model_name, results_dict in [("GPT-2 Small", all_results_gpt2), ("Gemma-2-2B", all_results_gemma)]:
    for method_name, results in results_dict.items():
        # Get results at alpha=5.0
        for r in results:
            if r['alpha'] == 5.0:
                rows.append({
                    'Model': model_name,
                    'Method': method_name,
                    'Probe Score Delta (alpha=5)': f"{r['probe_score_delta']:+.4f}",
                    'Steering Norm': f"{r['steering_norm']:.4f}",
                })

df = pd.DataFrame(rows)
print(df.to_string(index=False))

## Key Takeaways
- **CAAMeanDiff** uses the most direct signal (class-conditional means from labeled data)
- **CAAContrastive** doesn't require labeled data — just paired prompts
- **CAARepE** only needs neutral queries + persona descriptions
- All three produce unit-norm vectors; effectiveness depends on alpha scaling
- The three vectors often have high cosine similarity, suggesting they capture similar concept directions